# ENVPHYS200 Oceans Module

You can use this notebook to generate your plots.  Here we provide a nice example of the use of `pandas` and `geopandas` for you.  Very useful as geoscientists. 

In [1]:
!pip install ipyleaflet

import numpy as np
import matplotlib.pyplot as plt
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl, TileLayer, GeoData, FullScreenControl
import pandas as pd
import geopandas as gpd

# Download a data file from github, if it doesn't exist
!test ! -f SVP-example.dat && wget 'https://raw.githubusercontent.com/edur409/ENVPHYS/refs/heads/main/SVP-example.dat'

colnames=['Sample_number', 'Latitude', 'Longitude', 'E_V', 'N_V']
SVP = pd.read_csv('SVP-example.dat', delimiter=',', skiprows = 12 , skipfooter= 0, names = colnames, engine ='python', header = None)

# There's Pandas and Geopandas 

Dataframes and GeoDataframes are a nice way to organize data. 

In [2]:
# create a pandas Dataframe
df = pd.DataFrame(
    {'Sample_number': SVP.Sample_number,
     'E_V': SVP.E_V,
     'N_V': SVP.N_V,
     'Longitude': SVP.Longitude,
     'Latitude': SVP.Latitude})

# Transform it into GeoDataFrame with Geopandas
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Make it into a GeoData object for plotting fast in ipyleaflet 
geo_data = GeoData(geo_dataframe = gdf,
    style={'color': 'black', 'radius':2, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    point_style={'radius': 2, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
    name = 'Release')

# Ipyleaflet for plotting

A nice example of `ipyleaflet` capabilities.  The Geologic Map layer is available via the [Macrostrat.org](https://macrostrat.org/) API.  You can have a geologic map on your phone by installing their `Rockd` app. 

In [3]:
# Ipyleaflet example
center = (-38, -170)
zoom = 4

map = Map(center=center, zoom=zoom, interpolation='nearest')#, basemap= basemaps.OpenStreetMap)# basemaps.CartoDB.DarkMatter)

map.add(geo_data)
left_layer = TileLayer(url='https://tiles.macrostrat.org/carto/{z}/{x}/{y}.png')#basemap_to_tiles(basemaps.Esri.WorldStreetMap)
right_layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2000-10-02") # #basemaps.NASAGIBS.ModisTerraTrueColorCR

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)

map.add_control(control)
map.add(FullScreenControl())

display(map)

Map(center=[-38, -170], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

In [4]:
# plt.scatter(SVP.Longitude, SVP.Latitude)
# plt.axis('equal')